# Graph Transform Tool

https://petewarden.com/2016/12/30/rewriting-tensorflow-graphs-with-the-gtt/

https://github.com/tensorflow/tensorflow/tree/master/tensorflow/tools/graph_transforms

# Optimize Trained Models for Inference
## Types of Optimizations
* Remove training-only operations (checkpoint saving, drop out)
* Strip out unused nodes
* Remove debug operations
* Fold batch normalization ops into weights (super cool)
* Round weights
* Quantize weights

# Compare Types of Optimizations

## Original Model (CPU)

### File Size

In [ ]:
%%bash

ls -l /root/models/linear/cpu

### Graph

In [ ]:
%%bash

summarize_graph --in_graph=/root/models/optimize_me/unoptimized_cpu.pb

### Benchmarks

In [ ]:
%%bash

benchmark_model --graph=/root/models/optimize_me/unoptimized_cpu.pb \
    --input_layer=weights,bias,x_observed \
    --input_layer_type=float,float,float \
    --input_layer_shape=:: \
    --output_layer=add

## Strip Unused Nodes

In [ ]:
%%bash

transform_graph \
    --in_graph=/root/models/optimize_me/unoptimized_cpu.pb \
    --out_graph=/root/models/optimize_me/strip_unused_optimized_cpu.pb \
    --inputs='x_observed,weights,bias' \
    --outputs='add' \
    --transforms='strip_unused_nodes'

In [ ]:
%%bash

ls -l /root/models/optimize_me/

### Graph

In [ ]:
%%bash

summarize_graph --in_graph=/root/models/optimize_me/strip_unused_optimized_cpu.pb

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import re
from google.protobuf import text_format
from tensorflow.core.framework import graph_pb2

def convert_graph_to_dot(input_graph, output_dot, is_input_graph_binary):
    graph = graph_pb2.GraphDef()
    with open(input_graph, "rb") as fh:
        if is_input_graph_binary:
            graph.ParseFromString(fh.read())
        else:
            text_format.Merge(fh.read(), graph)
    with open(output_dot, "wt") as fh:
        print("digraph graphname {", file=fh)
        for node in graph.node:
            output_name = node.name
            print("  \"" + output_name + "\" [label=\"" + node.op + "\"];", file=fh)
            for input_full_name in node.input:
                parts = input_full_name.split(":")
                input_name = re.sub(r"^\^", "", parts[0])
                print("  \"" + input_name + "\" -> \"" + output_name + "\";", file=fh)
        print("}", file=fh)
        print("Created dot file '%s' for graph '%s'." % (output_dot, input_graph))
        

In [ ]:
input_graph='/root/models/optimize_me/strip_unused_optimized_cpu.pb'
output_dot='/root/notebooks/strip_unused_optimized_cpu.dot'
convert_graph_to_dot(input_graph=input_graph, output_dot=output_dot, is_input_graph_binary=True)

In [ ]:
%%bash

dot -T png /root/notebooks/strip_unused_optimized_cpu.dot \
    -o /root/notebooks/strip_unused_optimized_cpu.png > /tmp/a.out

In [ ]:
from IPython.display import Image

Image('/root/notebooks/strip_unused_optimized_cpu.png', width=1024, height=768)

### Benchmark

In [ ]:
%%bash

benchmark_model --graph=/root/models/optimize_me/strip_unused_optimized_cpu.pb \
    --input_layer=weights,bias,x_observed \
    --input_layer_type=float,float,float \
    --input_layer_shape=:: \
    --output_layer=add

### Fold Constants

In [ ]:
%%bash

transform_graph \
--in_graph=/root/models/optimize_me/unoptimized_cpu.pb \
--out_graph=/root/models/optimize_me/fold_constants_optimized_cpu.pb \
--inputs='x_observed' \
--outputs='add' \
--transforms='
fold_constants(ignore_errors=true)'

### File Size

In [ ]:
%%bash

ls -l /root/models/optimize_me/

### Graph

In [ ]:
%%bash

summarize_graph --in_graph=/root/models/optimize_me/fold_constants_optimized_cpu.pb

In [ ]:
input_graph='/root/models/optimize_me/fold_constants_optimized_cpu.pb'
output_dot='/root/notebooks/fold_constants_optimized_cpu.dot'
convert_graph_to_dot(input_graph=input_graph, output_dot=output_dot, is_input_graph_binary=True)

In [ ]:
%%bash

dot -T png /root/notebooks/fold_constants_optimized_cpu.dot \
    -o /root/notebooks/fold_constants_optimized_cpu.png > /tmp/a.out

In [ ]:
from IPython.display import Image

Image('/root/notebooks/fold_constants_optimized_cpu.png')

### Benchmarks

In [ ]:
%%bash

benchmark_model --graph=/root/models/optimize_me/fold_constants_optimized_cpu.pb \
                --input_layer=x_observed,bias,weights \
                --input_layer_type=float,float,float \
                --input_layer_shape=:: \
                --output_layer=add

## Fold Batch Normalizations
Must run Fold Constants first!

In [ ]:
%%bash

transform_graph \
--in_graph=/root/models/optimize_me/fold_constants_optimized_cpu.pb \
--out_graph=/root/models/optimize_me/fold_batch_norms_optimized_cpu.pb \
--inputs='x_observed' \
--outputs='add' \
--transforms='
fold_batch_norms
fold_old_batch_norms'

### File Size

In [ ]:
%%bash

ls -l /root/models/optimize_me/

### Graph

In [ ]:
%%bash

summarize_graph --in_graph=/root/models/optimize_me/fold_batch_norms_optimized_cpu.pb

In [ ]:
input_graph='/root/models/optimize_me/fold_batch_norms_optimized_cpu.pb'
output_dot='/root/notebooks/fold_batch_norms_optimized_cpu.dot'
convert_graph_to_dot(input_graph=input_graph, output_dot=output_dot, is_input_graph_binary=True)

In [ ]:
%%bash

dot -T png /root/notebooks/fold_batch_norms_optimized_cpu.dot \
    -o /root/notebooks/fold_batch_norms_optimized_cpu.png > /tmp/a.out

In [ ]:
from IPython.display import Image

Image('/root/notebooks/fold_batch_norms_optimized_cpu.png')

### Benchmarks

In [ ]:
%%bash

benchmark_model --graph=/root/models/optimize_me/fold_batch_norms_optimized_cpu.pb \
                --input_layer=x_observed,bias,weights \
                --input_layer_type=float,float,float \
                --input_layer_shape=:: \
                --output_layer=add

## Quantize Weights
Should run Fold Batch Normalization first!

In [ ]:
%%bash

transform_graph \
--in_graph=/root/models/optimize_me/fold_batch_norms_optimized_cpu.pb \
--out_graph=/root/models/optimize_me/quantize_weights_optimized_cpu.pb \
--inputs='x_observed' \
--outputs='add' \
--transforms='quantize_weights'

### File Size

In [ ]:
%%bash

ls -l /root/models/optimize_me/

### Graph

In [ ]:
%%bash

summarize_graph --in_graph=/root/models/optimize_me/quantize_weights_optimized_cpu.pb

In [ ]:
input_graph='/root/models/optimize_me/quantize_weights_optimized_cpu.pb'
output_dot='/root/notebooks/quantize_weights_optimized_cpu.dot'
convert_graph_to_dot(input_graph=input_graph, output_dot=output_dot, is_input_graph_binary=True)

In [ ]:
%%bash

dot -T png /root/notebooks/quantize_weights_optimized_cpu.dot \
    -o /root/notebooks/quantize_weights_optimized_cpu.png > /tmp/a.out

In [ ]:
from IPython.display import Image

Image('/root/notebooks/quantize_weights_optimized_cpu.png')

### Benchmarks

In [ ]:
%%bash

benchmark_model --graph=/root/models/optimize_me/quantize_weights_optimized_cpu.pb --input_layer=x_observed,bias,weights --input_layer_type=float,float,float --input_layer_shape=:: --output_layer=add

## Quantize Nodes

In [ ]:
%%bash

transform_graph \
--in_graph=/root/models/optimize_me/fold_batch_norms_optimized_cpu.pb \
--out_graph=/root/models/optimize_me/quantize_nodes_optimized_cpu.pb \
--inputs='x_observed' \
--outputs='add' \
--transforms='quantize_nodes'

In [ ]:
%%bash

ls -l /root/models/optimize_me/

In [ ]:
%%bash

summarize_graph --in_graph=/root/models/optimize_me/quantize_nodes_optimized_cpu.pb

In [ ]:
input_graph='/root/models/optimize_me/quantize_nodes_optimized_cpu.pb'
output_dot='/root/notebooks/quantize_nodes_optimized_cpu.dot'
convert_graph_to_dot(input_graph=input_graph, output_dot=output_dot, is_input_graph_binary=True)

In [ ]:
%%bash

dot -T png /root/notebooks/quantize_nodes_optimized_cpu.dot \
    -o /root/notebooks/quantize_nodes_optimized_cpu.png > /tmp/a.out

In [ ]:
from IPython.display import Image

Image('/root/notebooks/quantize_nodes_optimized_cpu.png')

### Benchmarks

In [ ]:
%%bash

benchmark_model --graph=/root/models/optimize_me/quantize_nodes_optimized_cpu.pb \
                --input_layer=x_observed,bias,weights \
                --input_layer_type=float,float,float \
                --input_layer_shape=:: \
                --output_layer=add

## Sort by Execution Order
* aka. Topological Order Sort
* Minimize inference overhead 
* Inputs for a each node are guaranteed to be available on the forward path

In [ ]:
%%bash

transform_graph \
--in_graph=/root/models/optimize_me/unoptimized_cpu.pb \
--out_graph=/root/models/optimize_me/sort_by_execution_order_optimized_cpu.pb \
--inputs='x_observed' \
--outputs='add' \
--transforms='
sort_by_execution_order'

### File Size

In [ ]:
%%bash

ls -l /root/models/optimize_me/

### Graph

In [ ]:
%%bash

summarize_graph --in_graph=/root/models/optimize_me/sort_by_execution_order_optimized_cpu.pb

### Benchmarks

In [ ]:
%%bash

benchmark_model --graph=/root/models/optimize_me/sort_by_execution_order_optimized_cpu.pb \
    --input_layer=x_observed,bias,weights \
    --input_layer_type=float,float,float \
    --input_layer_shape=:: \
    --output_layer=add

## Combine All Optimizations

In [ ]:
%%bash

transform_graph \
--in_graph=/root/models/optimize_me/unoptimized_cpu.pb \
--out_graph=/root/models/optimize_me/fully_optimized_cpu.pb \
--inputs='x_observed' \
--outputs='add' \
--transforms='
add_default_attributes
remove_nodes(op=Identity, op=CheckNumerics)
fold_constants(ignore_errors=true)
fold_batch_norms
fold_old_batch_norms
quantize_weights
quantize_nodes
strip_unused_nodes
sort_by_execution_order'

### File Size

In [ ]:
%%bash

ls -l /root/models/optimize_me/

### Graph

In [ ]:
%%bash

summarize_graph --in_graph=/root/models/optimize_me/fully_optimized_cpu.pb

In [ ]:
input_graph='/root/models/optimize_me/fully_optimized_cpu.pb'
output_dot='/root/notebooks/fully_optimized_cpu.dot'
convert_graph_to_dot(input_graph=input_graph, output_dot=output_dot, is_input_graph_binary=True)

In [ ]:
%%bash

dot -T png /root/notebooks/fully_optimized_cpu.dot \
    -o /root/notebooks/fully_optimized_cpu.png > /tmp/a.out

In [ ]:
from IPython.display import Image

Image('/root/notebooks/fully_optimized_cpu.png')

### Benchmarks

In [ ]:
%%bash

benchmark_model --graph=/root/models/optimize_me/fully_optimized_cpu.pb \
    --input_layer=weights,x_observed,bias \
    --input_layer_type=float,float,float \
    --input_layer_shape=:: \
    --output_layer=add

## Obfuscate Names
* Shorten and mangle internal graph node names

In [ ]:
%%bash

transform_graph \
--in_graph=/root/models/optimize_me/fully_optimized_cpu.pb \
--out_graph=/root/models/optimize_me/obfuscate_names_optimized_cpu.pb \
--inputs='x_observed' \
--outputs='add' \
--transforms='
obfuscate_names'

### File Size

In [ ]:
%%bash

ls -l /root/models/optimize_me/

### Graph

In [ ]:
%%bash

summarize_graph --in_graph=/root/models/optimize_me/obfuscate_names_optimized_cpu.pb